# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [218]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [219]:
X = boston_features[['B','DIS','LSTAT']]
y =  pd.DataFrame(boston.target, columns= ["price"])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [220]:
from sklearn.model_selection import train_test_split

In [221]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [222]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)


Calculate the residuals and the mean squared error

In [223]:
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

Train Mean Squarred Error: 26.092582081280565
Test Mean Squarred Error: 23.148261354062


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [241]:

def kfolds(data,k):
    t=len(data)%k-1
    size=len(data)//k
    #print('t',t)
    divdata=[]
    start=0
    end=size
    #print(len(data))
    for i in range(t):
        fold = data.iloc[[start, end], :]
        divdata.append(fold)
        start=end+1
        end=end+size+1
        #print(fold)
        #print('start',start)
        #print('end',end)
    while end<len(data)-1:
        fold = data.iloc[[start, end], :]
        divdata.append(fold)
        start=end+1
        end=end+size
        #print('start',start)
        #print('end',end)
        #print("after t",fold)

    return divdata
bostonfold

[            B       DIS     LSTAT  price
 0    1.000000  0.542096 -1.275260   24.0
 101  0.996672  0.390808 -0.555806   26.5,
             B       DIS     LSTAT  price
 102  0.177720  0.369415 -0.012136   18.6
 202  0.996167  0.722095 -2.059550   42.3,
             B       DIS     LSTAT  price
 203  0.989611  0.636563 -1.721374   48.5
 303  0.983686  0.666254 -1.315893   33.1,
             B       DIS     LSTAT  price
 304  0.991881  0.535033 -0.724816   36.1
 404  0.829946  0.148620  1.563971    8.5]

### Apply it to the Boston Housing Data

In [225]:
# Make sure to concatenate the data again
bostonxy=pd.concat([X.reset_index(drop=True),y], axis=1)
bostonfold = kfolds(bostonxy,5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [226]:
test_errs = []
train_errs = []

k=5

for n in range(k):
    # Split in train and test for the fold

    train = pd.concat([fold for i, fold in enumerate(bostonfold) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)


[price    7.562406
dtype: float64, price    39.113035
dtype: float64, price    14.336995
dtype: float64, price    29.932076
dtype: float64, price    30.031782
dtype: float64]
[price    508.858665
dtype: float64, price    78.837635
dtype: float64, price    499.293172
dtype: float64, price    141.219525
dtype: float64, price    21.514427
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [233]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results =cross_val_score(linreg, X,y, cv=5, scoring="neg_mean_squared_error")
cv_5_results




array([-19.51466367, -19.33713985, -49.27049788, -37.65135885,
       -17.91502022])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [242]:
cv_5_results = np.mean(cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error"))

print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)


print ('cross validation mse:', cv_5_results)

#in my test, cros validation result came out worse than train-test split case. 
# for this model it look better to use train-test split method.   


Train Mean Squarred Error: 26.092582081280565
Test Mean Squarred Error: 23.148261354062
cross validation mse: -28.737736096449662


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!